In [1]:
import os

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [4]:
LANGCHAIN_PROJECT

'langgraph-prerequist'

In [5]:

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [31]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

In [7]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name="Gemma2-9b-It")

# Simple AI Assistant

In [11]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Hello, Sunny. I am Gemini, a multi-modal AI model, developed by Google. How can I assist you today?
Hi Sunny, it's nice to meet you! How can I help you today?
I do not have access to your personal information, including your name.
I do not have access to your personal information, so I cannot tell you your name.
goodbye take care yourself


In [12]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [20]:
store={}

In [21]:

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [22]:
config = {"configurable": {"session_id": "firstchat"}}

In [23]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [24]:
model_with_memory.invoke(("Hi! I'm sunnysavita"),config=config).content

"Hello, sunnysavita! It's nice to meet you. My name is Gemini. I am a multi-modal AI model, developed by Google. I am designed to assist and provide information to the best of my abilities. Is there anything I can help you with today?"

In [25]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is sunnysavita. You told me this when you introduced yourself. Is there anything else I can help you with today?'

In [26]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm sunnysavita", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello, sunnysavita! It's nice to meet you. My name is Gemini. I am a multi-modal AI model, developed by Google. I am designed to assist and provide information to the best of my abilities. Is there anything I can help you with today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-fc4908ce-f4a1-4f0f-aab4-1914f82593b8-0', usage_metadata={'input_tokens': 10, 

# RAG with LCEL

In [30]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [32]:
### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [33]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)


In [34]:

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [35]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Based on the provided context, here are three important points about Llama3:

1. **Llama3 is a language model:** It is a large language model developed by Meta AI.
2. **It was released in April 2024:** Llama3 was released in April 2024, according to the provided context.
3. **It is used by various services:** Both Notion AI and Gemini use a Llama3 model, as stated in the context.


# Let's Start with Tools and Agents

In [36]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [37]:
api_wrapper=WikipediaAPIWrapper()

In [38]:
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [39]:
tool.name

'wikipedia'

In [40]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [41]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [42]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Retrieval-augmented generation
Summary: Retrieval augmented generation (RAG) is a type of generative artificial intelligence that has information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information in preference to information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  
Use cases include providing chatbot access to internal company data, or giving factual information only from an authoritative source.



Page: DataStax
Sum

In [43]:
from langchain_community.tools import YouTubeSearchTool

In [44]:
tool2=YouTubeSearchTool()

In [45]:
tool2.name

'youtube_search'

In [47]:
tool2.run("sunny savita")

"['https://www.youtube.com/watch?v=V9tJCQoBakA&pp=ygUMc3Vubnkgc2F2aXRh', 'https://www.youtube.com/watch?v=6GnsndNL5u4&pp=ygUMc3Vubnkgc2F2aXRh']"

In [48]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [49]:
tool3=TavilySearchResults()

In [50]:
tool.invoke({"query": "What happened in the latest burning man floods"})

'Page: Burning Man\nSummary: Burning Man is a week-long large-scale desert event focused on "community, art, self-expression, and self-reliance" held annually in the western United States. The event\'s name comes from its culminating ceremony: the symbolic burning of a large wooden effigy, referred to as the Man, that occurs on the penultimate night, the Saturday evening before Labor Day. Since 1991, the event has been at Black Rock City in northwestern Nevada, a temporary city erected in the Black Rock Desert about 100 miles (160 km) north-northeast of Reno. According to Burning Man co-founder Larry Harvey in 2004, the event is guided by ten stated principles: radical inclusion, gifting, decommodification, radical self-reliance, radical self-expression, communal effort, civic responsibility, leaving no trace, participation, and immediacy.\nBurning Man features no headliners or scheduled performers; instead, participants create all the art, activities, and events. Artwork includes expe

In [51]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [60]:
tool=load_tools(["wikipedia"],llm=llm)

In [61]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [62]:
agent.run("What is current GDP of India?")



> Entering new AgentExecutor chain...
Action: [wikipedia]
Action Input: GDP of india
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: India GDP
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: India GDP
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: India GDP
Observation: [wikipedia] is not a valid tool, t

'Agent stopped due to iteration limit or time limit.'